In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
#reading data including cluster output
data_tr=pd.read_csv("transaction_data.csv",low_memory=False)
data_pr=pd.read_csv("product.csv",low_memory=False)
labels=pd.read_csv("output_kmeans_labels.csv")
cluster_description=pd.read_csv("Segment description.csv",sep=";")

In [3]:
data_tr_=data_tr.merge(labels,how="left")
#remove unlabeled households (~1%)
data_tr_=data_tr_[~data_tr_.clusters.isna()]

In [4]:
#data for dashboard preparation 
data_tr_clearw = data_tr_
bins_trans_time = [0, 600, 900, 1800, 2400]
group_names = ["night","morning","day","evening"]
data_tr_clearw = data_tr_clearw.assign(TRANS_TIME_cat=(pd.cut(data_tr_clearw['TRANS_TIME'], bins_trans_time, labels=group_names)))
data_tr_clearw_short=data_tr_clearw[["household_key", "PRODUCT_ID","SALES_VALUE","TRANS_TIME_cat","WEEK_NO"]]
data_tr_clearw_short_pr=data_tr_clearw_short.merge(data_pr,how="left").dropna()
data_tr_clearw_short_pr.loc[data_tr_clearw_short_pr.DEPARTMENT==' ','DEPARTMENT'] = "OTHER"
data_tr_clearw_short_pr_=data_tr_clearw_short_pr[["household_key", "SALES_VALUE","TRANS_TIME_cat","DEPARTMENT","WEEK_NO"]]
data_tr_clearw_short_pr__=data_tr_clearw_short_pr_.merge(labels,how="left")
data_chart1=pd.DataFrame(data_tr_clearw_short_pr__.groupby(["WEEK_NO","clusters","DEPARTMENT","household_key"])["SALES_VALUE"].sum()).reset_index()

In [5]:
#filter out departments with low sales in last 12 weeks (25 of total sales)
dep_sales=data_tr_clearw_short_pr__[data_tr_clearw_short_pr__.WEEK_NO>90].groupby("DEPARTMENT")["SALES_VALUE"].sum()
list_of_dep=list(dep_sales[dep_sales>10000].index)

In [9]:
#this returns moving average with NMA window. For first values it takes its mean
def moving_average_n(x,NMA):
    a=[]
    for z in range(NMA-1):
        a.append(np.array(x[:(z+1)]).mean())
    return np.concatenate([a,np.convolve(x, np.ones(NMA)/NMA, mode='full')[(NMA-1):-(NMA-1)]])

In [10]:
def format_yoy(arr,ind_arr,arr2):
    try:
        a=arr[-1]
        d=arr2[-1]
        b=arr[ind_arr[-1]-53]
        c=arr2[ind_arr[-1]-53]
        total_growth_abs = a*d-b*c
        total_growth_rel = (a*d)/(b*c)-1
        spendings_growth_rel = a/b-1
        households_growth_rel = d/c-1
        return "{:.1%}".format(total_growth_rel),str(int(total_growth_abs))+"$","{:.1%}".format(spendings_growth_rel),"{:.1%}".format(households_growth_rel),total_growth_rel 
    except (IndexError,ValueError): return ["NA"]*5

In [11]:
def print_yoy(df_spendings, df_spendings_index, df_households):    
    total_growth_rel,total_growth_abs,spending_growth_rel,households_growth_rel,total_growth_rel_unf=format_yoy(df_spendings, df_spendings_index, df_households)             
    print(' YoY total growth:',total_growth_rel,",",total_growth_abs)
    print(' YoY spendings per households growth:',spending_growth_rel)
    print(' YoY households growth:',households_growth_rel)

In [13]:
NMA=4 #moving average window
total_clusters=7
last_week=data_chart1.groupby("WEEK_NO")["SALES_VALUE"].sum().index[-1]
total_size = len(data_chart1[data_chart1.WEEK_NO==last_week].household_key.unique())

In [14]:
def f(Cluster, Department):
    cl = Cluster
    pr = Department
    if cl != "ALL" and pr != "ALL": tmp1 = data_chart1[(data_chart1.clusters==cl)&(data_chart1.DEPARTMENT==pr)]
    else:                           tmp1 = data_chart1
    if cl != "ALL": tmp2 = data_chart1[(data_chart1.clusters==cl)]
    if pr != "ALL": tmp3 = data_chart1[(data_chart1.DEPARTMENT==pr)]
    if cl=="ALL": 
        l = total_size
        descr="all households"
    else:         
        l = len(tmp2[tmp2.WEEK_NO==last_week].household_key.unique())
        descr=cluster_description.loc[cluster_description.cluster==cl,"description"].iloc[0]
    print('Cluster description:',descr)    
    print('Size of cluster =',l,",","{:.0%}".format(l/total_size), " of total" )
    
    #out1 - filter by cluster and product
    #out2 - filter by product
    #out3 - (out1-out2)/out2 , shows cluster perfomance
    if len(tmp1) > 0:
        if pr == "ALL" and cl == "ALL":   
            households=tmp1.groupby("WEEK_NO")["household_key"].nunique()
            out1=tmp1.groupby("WEEK_NO")["SALES_VALUE"].sum() / households
            households2=households
            out2 = out1
        elif pr == "ALL":    
            households=tmp2.groupby("WEEK_NO")["household_key"].nunique()
            out1=tmp2.groupby("WEEK_NO")["SALES_VALUE"].sum() / households
            households2=tmp1.groupby("WEEK_NO")["household_key"].nunique()
            out2=(tmp1.groupby("WEEK_NO")["SALES_VALUE"].sum() / households2)
        elif cl == "ALL":
            households=tmp3.groupby("WEEK_NO")["household_key"].nunique()
            out1=tmp3.groupby("WEEK_NO")["SALES_VALUE"].sum() / households
            households2=households
            out2=out1
        else:
            households=tmp1.groupby("WEEK_NO")["household_key"].nunique()
            out1=tmp1.groupby("WEEK_NO")["SALES_VALUE"].sum() / households
            households2=tmp3.groupby("WEEK_NO")["SALES_VALUE"].nunique()
            out2=(tmp3.groupby("WEEK_NO")["SALES_VALUE"].sum() / households2)
        out3 = 100 * (out1/out2 - 1)
        
        #moving average for charts and YoY calc
        households_MA=moving_average_n(households,NMA)
        households2_MA=moving_average_n(households2,NMA)
        out1_MA=moving_average_n(out1,NMA)
        out3_MA=moving_average_n(out3,NMA)
        out2_MA=moving_average_n(out2,NMA)
        
        #print perfomance metrics
        print("For selected cluster:")
        print_yoy(out1_MA,out1.index, households_MA)       
        print("For all clusters:")
        print_yoy(out2_MA,out2.index, households2_MA)

        #data for cluster spendings pie
        cl_list=[]
        pie_values_list=[]
        for c in list(range(0,total_clusters)):
            cl_list.append(c)
            if pr!="ALL": tmp_cl=data_chart1[(data_chart1.clusters==c)&(data_chart1.DEPARTMENT==pr)].groupby("WEEK_NO")["SALES_VALUE"].sum()
            else:         tmp_cl=data_chart1[(data_chart1.clusters==c)].groupby("WEEK_NO")["SALES_VALUE"].sum()
            
            try: pie_value=tmp_cl[last_week]
            except (KeyError, IndexError): pie_value=0
            pie_values_list.append(pie_value)
        labels_pie1 = cl_list
        values_pie1 = [round(m / sum(pie_values_list),3) for m in pie_values_list]
        
        #data for households pie
        pie_values_list=[]
        for c in list(range(0,total_clusters)):
            if pr!="ALL": tmp_cl=data_chart1[(data_chart1.clusters==c)&(data_chart1.DEPARTMENT==pr)].groupby("WEEK_NO")["household_key"].nunique()
            else:         tmp_cl=data_chart1[(data_chart1.clusters==c)].groupby("WEEK_NO")["household_key"].nunique()
            try: pie_value=tmp_cl[last_week]
            except (KeyError, IndexError): pie_value=0
            pie_values_list.append(pie_value)
        values_pie2 = [round(m / sum(pie_values_list),3) for m in pie_values_list]
        
        #data for departments pie
        pr_list=[]
        pie_values_list=[]
        i=0
        tmp_i=0
        for p in list_of_dep:
            pr_list.append(p)
            if cl!="ALL": tmp_cl=data_chart1[(data_chart1.clusters==cl)&(data_chart1.DEPARTMENT==p)].groupby("WEEK_NO")["SALES_VALUE"].sum()
            else:         tmp_cl=data_chart1[(data_chart1.DEPARTMENT==p)].groupby("WEEK_NO")["SALES_VALUE"].sum()
            try: pie_value=tmp_cl[last_week]
            except (KeyError, IndexError): pie_value=0
            pie_values_list.append(pie_value)
            if p==pr: tmp_i=i
            i+=1
        labels_pie3 = pr_list
        values_pie3 = [round(m / sum(pie_values_list),4) for m in pie_values_list]
        
        #data for bar chart
        cl_list=[]
        bar_values_list=[]
        ii=0
        tmp_ii=0
        for c in list(range(0,total_clusters)):
            cl_list.append(c)
            if pr!="ALL": tmp_cl=data_chart1[(data_chart1.clusters==c)&(data_chart1.DEPARTMENT==pr)]
            else:         tmp_cl=data_chart1[(data_chart1.clusters==c)]
            households_tmp=tmp_cl.groupby("WEEK_NO")["household_key"].nunique()
            out1_tmp=tmp_cl.groupby("WEEK_NO")["SALES_VALUE"].sum() / households_tmp
            households_MA_tmp=moving_average_n(households_tmp,NMA)
            out1_MA_tmp=moving_average_n(out1_tmp,NMA)
            Yoy_list=format_yoy(out1_MA_tmp,out1_tmp.index, households_MA_tmp)
            bar_value=Yoy_list[4]
            try: bar_values_list.append(round(bar_value,3)*100)
            except (TypeError): bar_values_list.append("NA")            
            if c==cl: tmp_ii=ii
            ii+=1    
        values_bar = bar_values_list
        
        #rendering dashboard
        xy1_title="Number of households in selected cluster" #with at least one transaction of selected product
        xy2_title="Spendings of selected cluster per household"
        xy3_title="Diff spendings of cluster per household" #compared with spendings of all clusters
        xy4_title="Spendings of all clusters per household"
        
        pie1_title="Spendings split by cluster in last week"
        pie2_title="Households split by cluster in last week"
        pie3_title="Spendings split by department in last week"
        fig = make_subplots(rows=4, cols=2,
                            specs=[[{'type':'xy'}, {'type':'domain'}],
                                   [{'type':'xy'}, {'type':'domain'}],
                                   [{'type':'xy'}, {'type':'domain'}],
                                   [{'type':'xy'}, {'type':'xy'}]],
                            #subplot_titles=("Number of households","Spendings of selected cluster per household compared with all clusters","Spendings of all clusters per household")
                            shared_xaxes=True,vertical_spacing=0.07,horizontal_spacing=0.05,
                            subplot_titles=(xy1_title, pie2_title, 
                                           xy2_title, pie1_title,
                                           xy3_title, pie3_title,
                                           xy4_title))
        
        fig.append_trace(
            go.Scatter(x=list(households.index), 
                       y=list(households),
                       name="Number of households in selected cluster with at least one transaction of selected product",
                       showlegend=False
                       ),row=1,col=1)
        fig.append_trace(
            go.Scatter(x=list(households.index), 
                       y=households_MA,
                       name="Moving_average(4) of Number of households in selected cluster with at least one transaction of selected product",
                       showlegend=False
                       ),row=1,col=1)
        fig.append_trace(
            go.Scatter(x=list(out1.index), 
                       y=list(out1),
                       name="Spendings of selected cluster per household",
                       showlegend=False
                       ),row=2,col=1)
        fig.append_trace(
            go.Scatter(x=list(out1.index), 
                       y=out1_MA,
                       name="Moving_average(4) of Spendings of selected cluster per household",
                       showlegend=False
                       ),row=2,col=1)
        fig.append_trace(
            go.Scatter(x=list(out3.index), 
                       y=list(out3),
                       name="Spendings of selected cluster per household compared with spendings of all clusters",
                       showlegend=False
                       ),row=3,col=1)
        fig.append_trace(
            go.Scatter(x=list(out3.index), 
                       y=out3_MA,
                       name="Moving_average(4) of Spendings of selected cluster per household compared with spendings of all clusters",
                       showlegend=False
                       ),row=3,col=1)
        fig.append_trace(
            go.Scatter(x=list(out2.index), 
                       y=list(out2),
                       name="Spendings of all clusters per household",
                       showlegend=False
                       ),row=4,col=1)
        fig.append_trace(
            go.Scatter(x=list(out2.index), 
                       y=out2_MA,
                       name="Moving_average(4) of Spendings of all clusters per household",
                       showlegend=False
                       ),row=4,col=1)
        

        if cl=="ALL": 
            fig_pie1=go.Pie(labels=labels_pie1, values=values_pie1, name=pie1_title,showlegend=False)
            fig_pie2=go.Pie(labels=labels_pie1, values=values_pie2, name=pie2_title,showlegend=False)
        else:
            tmp_pull=np.zeros(total_clusters)
            tmp_pull[cl]=0.2
            fig_pie1=go.Pie(labels=labels_pie1, values=values_pie1, name=pie1_title,showlegend=False,pull=tmp_pull)
            fig_pie2=go.Pie(labels=labels_pie1, values=values_pie2, name=pie2_title,showlegend=False,pull=tmp_pull)
        if pr=="ALL":
            fig_pie3=go.Pie(labels=labels_pie3, values=values_pie3, name=pie3_title,showlegend=False,)
        else:            
            tmp_pull=np.zeros(len(list_of_dep))
            tmp_pull[tmp_i] = 0.2
            fig_pie3=go.Pie(labels=labels_pie3, values=values_pie3, name=pie3_title,showlegend=False,pull=tmp_pull)
        
        #color selection on bar chart
        colors = ['orange',] * 7
        if cl!="ALL": colors[tmp_ii] = 'blue'
        fig_bar=go.Bar(
            y=labels_pie1,
            x=values_bar*100,
            orientation='h',
            name="YoY total growth",
            marker_color=colors,
            showlegend=False
        )
                
        fig.append_trace(fig_pie2,row=1,col=2)    
        fig.append_trace(fig_pie1,row=2,col=2)    
        fig.append_trace(fig_pie3,row=3,col=2)    
        fig.append_trace(fig_bar,row=4,col=2)    

        fig.update_layout(height=1100, width=1000,
                          #legend=dict(x=-.1, y=1.2),
                          hovermode="x",                          
                         )
        fig.update_yaxes(title_text="households in cluster", row=1, col=1)
        fig.update_yaxes(title_text="$, per household in cluster", row=2, col=1)
        fig.update_yaxes(title_text="%, diff", row=3, col=1)
        fig.update_yaxes(title_text="$, per household for all clusters", row=4, col=1)
        fig.update_xaxes(title_text="time, weeks", row=4, col=1)
        fig.update_xaxes(title_text="YoY total growth by cluster, %", row=4, col=2)
        fig.update_yaxes(type="category",row=4, col=2)
        
        fig.show()
    else:
        print("NO DATA")

In [113]:
interactive_plot = interactive(f, Cluster = ["ALL"] + list(range(0,7)), Department = ["ALL"] + list_of_dep)
#output = interactive_plot.children[-1]
#output.layout.height = '1300px'
interactive_plot

interactive(children=(Dropdown(description='Cluster', options=('ALL', 0, 1, 2, 3, 4, 5, 6), value='ALL'), Drop…